In [ ]:
include("../src/ProcessSequence.jl")

In [ ]:
cdna_fasta_gz = "/Volumes/CarrotCake/garden/data/grch/Homo_sapiens.GRCh38.cdna.all.fa.gz"

n_job = 8

In [ ]:
for _1_fastq_gz in readdir("/Users/kwat/Downloads/"; join = true)

    if occursin("_R1", _1_fastq_gz)

        _2_fastq_gz = replace(_1_fastq_gz, "R1" => "R2")

        output_dir = string(split(_1_fastq_gz, "_")[2])

        ProcessSequence.count_transcript(
            _1_fastq_gz,
            _2_fastq_gz,
            cdna_fasta_gz,
            output_dir,
            n_job::Int64,
        )

    end

end

## Make transcript by sample

In [ ]:
from environment import *

with open("setting.json") as io:

    SETTING = json.load(io)

PATH = make_path_dict(SETTING)

In [ ]:
enst_tpms = []

for sample_id in SETTING["fastq_id_fastq_file_paths"].keys():

    enst_tpm = pd.read_csv(
        os.path.join(PATH["kallisto/"], sample_id, "abundance.tsv"),
        sep="\t",
        index_col=0,
    )["tpm"]

    enst_tpm.name = sample_id

    enst_tpms.append(enst_tpm)

enst_x_sample = pd.concat(enst_tpms, axis=1)

enst_x_sample.index.name = "ENST"

enst_x_sample.to_csv(PATH["enst_x_sample.tsv"], sep="\t")

## Make gene by sample

In [ ]:
enst_gene_name = pd.read_csv(SETTING["enst_gene_name_file_path"], sep="\t")

gene_x_sample = pd.DataFrame(
    enst_x_sample,
    index=enst_x_sample.index.map(
        {
            enst: gene_name.str.upper()
            for enst, gene_name in zip(
                enst_gene_name["Transcript stable ID version"],
                enst_gene_name["Gene name"],
            )
        }
    ),
)

gene_x_sample.loc[~gene_x_sample.index.isna()].groupby(level=0).median()

gene_x_sample.index.name = "Gene"

gene_x_sample.to_csv(PATH["gene_x_sample.tsv"], sep="\t")